# Convolutional Neural Network

**A Convolutional Neural Network (ConvNet/CNN) is a Deep Learning algorithm which can take in an input image, assign importance (learnable weights and biases) to various aspects/objects in the image and be able to differentiate one from the other. The pre-processing required in a ConvNet is much lower as compared to other classification algorithms. While in primitive methods filters are hand-engineered, with enough training, ConvNets have the ability to learn these filters/characteristics.**

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from string import Template
import IPython.display
import warnings 

# Convolution Layer

In [280]:
class Conv:
    
    def __init__(self, num_filters):
        self.num_filters = num_filters
        
        #why divide by 9...Xavier initialization
        self.filters = np.random.randn(num_filters, 3, 3)/9
    
    def iterate_regions(self, image):
        h,w = image.shape

        #generates all possible 3*3 image regions using valid padding
        
        for i in range(h-2):
            for j in range(w-2):
                im_region = image[i:(i+3), j:(j+3)]
                yield im_region, i, j
                
    def forward(self, input):
        self.last_input = input
        
        h,w = input.shape
        output = np.zeros((h-2, w-2, self.num_filters))
        
        for im_regions, i, j in self.iterate_regions(input):
            output[i, j] = np.sum(im_regions * self.filters, axis=(1,2))
        return output
    
    def backprop(self, d_l_d_out, learn_rate):
        '''
        Performs a backward pass of the conv layer.
        - d_L_d_out is the loss gradient for this layer's outputs.
        - learn_rate is a float.
        '''
        d_l_d_filters = np.zeros(self.filters.shape)

        for im_region, i, j in self.iterate_regions(self.last_input):
            for f in range(self.num_filters):
                d_l_d_filters[f] += d_l_d_out[i,j,f] * im_region

        #update filters
        self.filters -= learn_rate * d_l_d_filters

        return None

# MaxPooling Layer

*A Max Pooling layer can’t be trained because it doesn’t actually have any weights, but we still need to implement a backprop() method for it to calculate gradients. We’ll start by adding forward phase caching again. All we need to cache this time is the input:*

*During the forward pass, the Max Pooling layer takes an input volume and halves its width and height dimensions by picking the max values over 2x2 blocks. The backward pass does the opposite: we’ll double the width and height of the loss gradient by assigning each gradient value to where the original max value was in its corresponding 2x2 block.*

In [281]:
class MaxPool:
    def iterate_regions(self, image):
        h, w, _ = image.shape
        
        # floor division
        new_h = h // 2
        new_w = w // 2
        
        for i in range(new_h):
            for j in range(new_w):
                im_region = image[(i*2):(i*2+2), (j*2):(j*2+2)]
                yield im_region, i, j
                
    def forward(self, input):
        
        self.last_input = input
        
        h, w, num_filters = input.shape
        output = np.zeros((h//2, w//2, num_filters))
        
        for im_region, i, j in self.iterate_regions(input):
            output[i,j] = np.amax(im_region,axis=(0,1))
            
        return output
    
    def backprop(self, d_l_d_out):
        '''
        Performs a backward pass of the maxpool layer.
        Returns the loss gradient for this layer's inputs.
        - d_L_d_out is the loss gradient for this layer's outputs.
        '''
        d_l_d_input = np.zeros(self.last_input.shape)

        for im_region, i, j in self.iterate_regions(self.last_input):
            h, w, f = im_region.shape
            amax = np.amax(im_region, axis=(0,1))

            for i2 in range(h):
                for j2 in range(w):
                    for f2 in range(f):
                        #if the pixel was the max value, copy the gradient to it
                        if(im_region[i2,j2,f2] == amax[f2]):
                            d_l_d_input[i*2+i2, j*2+j2 ,f2] = d_l_d_out[i, j, f2]
                            break;
        return d_l_d_input

# Softmax Layer

In [282]:
class Softmax:
    def __init__(self, input_len, nodes):
        # We divide by input_len to reduce the variance of our initial values
        self.weights = np.random.randn(input_len, nodes)/input_len
        self.biases = np.zeros(nodes)
    
    def forward(self, input):
        
        self.last_input_shape = input.shape
        
        input = input.flatten()
        self.last_input = input
        
        input_len, nodes = self.weights.shape
        
        totals = np.dot(input, self.weights) + self.biases
        self.last_totals = totals
        
        exp = np.exp(totals)
        return(exp/np.sum(exp, axis=0)) 
    
    def backprop(self, d_l_d_out, learn_rate):
        """  
        Performs a backward pass of the softmax layer.
        Returns the loss gradient for this layers inputs.
        - d_L_d_out is the loss gradient for this layers outputs.
        """
        
        #We know only 1 element of d_l_d_out will be nonzero
        for i, gradient in enumerate(d_l_d_out):
            if(gradient == 0):
                continue
            
            #e^totals
            t_exp = np.exp(self.last_totals)
            
            #Sum of all e^totals
            S = np.sum(t_exp)
            
            #gradients of out[i] against totals
            d_out_d_t = -t_exp[i] * t_exp/ (S**2)
            d_out_d_t[i] = t_exp[i] * (S-t_exp[i]) /(S**2)
            
            # Gradients of totals against weights/biases/input
            d_t_d_w = self.last_input
            d_t_d_b = 1
            d_t_d_inputs = self.weights
            
            #Gradients of loss against totals
            d_l_d_t = gradient * d_out_d_t
            
            #Gradients of loss against weights/biases/input
            d_l_d_w = d_t_d_w[np.newaxis].T @ d_l_d_t[np.newaxis]
            d_l_d_b = d_l_d_t * d_t_d_b  
            d_l_d_inputs = d_t_d_inputs @ d_l_d_t
            
            #update weights/biases
            self.weights -= learn_rate * d_l_d_w
            self.biases -= learn_rate * d_l_d_b
            return d_l_d_inputs.reshape(self.last_input_shape)

In [324]:
# data = pd.read_csv('digit-recognizer/train.csv')

# X_train = data.iloc[:, 1:785].T / 255.
# Y_train = data.T.iloc[1:785, 0]

# X_train = np.array(X_train)
# Y_train = np.array(Y_train)

# np.random.shuffle(X_train) # shuffle before splitting into dev and training sets
# np.random.shuffle(Y_train) # shuffle before splitting into dev and training sets

# # data_dev = data[0:1000].T
# # Y_dev = data_dev[0]
# # X_dev = data_dev[1:n]
# # X_dev = X_dev / 255.

# # data_train = data[1000:m].T
# # Y_train = data_train[0]
# # X_train = data_train[1:n]
# # X_train = X_train / 255.
# print(Y_train.shape)
# X_train.shape

(784,)


(784, 42000)

In [19]:
data = pd.read_csv('digit-recognizer/train.csv')
data = np.array(data)

datatrain = data[0:1000].T
Y = datatrain[0]
Xtrain = datatrain[1:1000]
Xtrain = Xtrain / 255.
print(datatrain)
print(Y.shape)
Xtrain[0].shape

datadev = data[1001:2001].T
Ydev = datadev[0]
Xdev = datadev[1:1000] / 255.
print(Xdev.shape)
Xdev[0].shape

[[1 0 1 ... 9 6 4]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(1000,)
(784, 1000)


(1000,)

# Training

In [9]:
conv = Conv(8)
pool = MaxPool()
softmax = Softmax(13 * 13 * 8, 10)

def forward(image, label):
    # We transform the image from [0, 255] to [-0.5, 0.5] to make it easier
    # to work with. This is standard practice.
    
    out = conv.forward((image) - 0.5)
    out = pool.forward(out)
    out = softmax.forward(out)
    
    #calculate cross-entropy loss and accuracy
    loss = -np.log(out[label])
    acc = 1 if(np.argmax(out) == label) else 0
    
    return out, loss, acc

def train(im, label, lr=0.005):
    #forward
    im.shape
    out,loss,acc = forward(im, label)
     
    #calculate initial gradient
    gradient = np.zeros(10)
    gradient[label] = -1/out[label]
    
    
    #Backprop
    gradient = softmax.backprop(gradient, lr)
    gradient = pool.backprop(gradient)
    gradient = conv.backprop(gradient, lr)
    
    return np.argmax(out, 0), loss, acc
    
    
print('MNIST CNN initialized')

for epoch in range(3):
    print('----EPOCH %d ---'%(epoch+1))
    
    #shuffle the training data
    permutation = np.random.permutation(len(X_train))
    X_train = X_train[permutation]
    Y_train = Y_train[permutation]

    loss = 0
    num_correct = 0

    for i, (im, label) in enumerate(zip(Xtrain, Y)):
        #print stats every 100 steps
        if(i>0 and i %100 == 99):
            print('[Step %d] Past 100 steps: Average Loss %.3f | Accuracy: %d%%' %(i + 1, loss / 100, num_correct))

            loss = 0
            num_correct = 0
        im = im.reshape((28, 28))
        out, l, acc = train(im, label)
        loss += l
        num_correct += acc
        

MNIST CNN initialized
----EPOCH 1 ---
[Step 100] Past 100 steps: Average Loss 2.321 | Accuracy: 12%
[Step 200] Past 100 steps: Average Loss 2.360 | Accuracy: 4%
[Step 300] Past 100 steps: Average Loss 2.337 | Accuracy: 8%
[Step 400] Past 100 steps: Average Loss 2.344 | Accuracy: 12%
[Step 500] Past 100 steps: Average Loss 2.299 | Accuracy: 18%
[Step 600] Past 100 steps: Average Loss 2.321 | Accuracy: 12%
[Step 700] Past 100 steps: Average Loss 2.346 | Accuracy: 9%
----EPOCH 2 ---
[Step 100] Past 100 steps: Average Loss 2.305 | Accuracy: 8%
[Step 200] Past 100 steps: Average Loss 2.332 | Accuracy: 6%
[Step 300] Past 100 steps: Average Loss 2.303 | Accuracy: 14%
[Step 400] Past 100 steps: Average Loss 2.314 | Accuracy: 13%
[Step 500] Past 100 steps: Average Loss 2.272 | Accuracy: 19%
[Step 600] Past 100 steps: Average Loss 2.292 | Accuracy: 12%
[Step 700] Past 100 steps: Average Loss 2.328 | Accuracy: 10%
----EPOCH 3 ---
[Step 100] Past 100 steps: Average Loss 2.300 | Accuracy: 7%
[Step 

In [20]:
print('Testing the CNN')
loss = 0
num_correct = 0
for im, label in zip(Xdev.T, Ydev.T):
  im = im.reshape((28, 28))
  _, l, acc = forward(im, label)
  loss += l
  num_correct += acc

num_tests = len(Xdev)
print('Test Loss:', loss / num_tests)
print('Test Accuracy:', num_correct / num_tests)


Testing the CNN
Test Loss: 2.9377489138603443
Test Accuracy: 0.11352040816326531
